In [1]:
import sympy as sym 
import math
import cmath
import cython as cp
import numpy as np
import pytearcat as pt
from sympy import *
from sympy.physics import *
from sympy.tensor.tensor import *
from sympy import *
from sympy.diffgeom import *
from sympy.diffgeom.rn import *
import numpy as np
from itertools import permutations

In [2]:
t, x, y, z = symbols('t x y z', real=True)

TP = TensorProduct
tp = tensorproduct
tc = tensorcontraction

GR4 = Manifold('R^4', 4)
GR4_origin = Patch('origin', GR4)
GR4_r = CoordSystem('rectangular', GR4_origin, symbols('t x y z', real=True))

t, x, y, z,  = [ Dummy (s) for s in
               ['t', 'x', 'y', 'z']]

In [3]:
GR4_r.t, GR4_r.x, GR4_r.y, GR4_r.z = GR4_r.coord_functions()

GR4_r.e_t, GR4_r.e_x, GR4_r.e_y, GR4_r.e_z = GR4_r.base_vectors()

GR4_r.dt, GR4_r.dx, GR4_r.dy, GR4_r.dz = GR4_r.base_oneforms()

t, x, y, z = GR4_r.symbols

In [4]:
def lower_indices (T, expr, changes):
    metric = twoform_to_matrix(expr)

    N = T.rank ()
    lis = list (range(len(changes), N))
    em = []
    k = len(changes)-1
    l = 0

    for j in range (N) :
        if j in changes :
            em.append (k)
            k -= 1
        else:
            em.append(lis[l])
            l += 1

    new_order = tuple (em)

    lowered = T

    for i in changes :
         lowered = tc(tp(metric, lowered), (0, i +2))

    lowered = permutedims(lowered, new_order)

    return lowered

def raise_indices (T, expr, changes):
    coord_sys = expr.atoms(CoordSystem).pop()
    indices = list(range(coord_sys.dim))
    matrix = twoform_to_matrix(expr)
    s_fields = set()
    for e in matrix :
        s_fields.update (e.atoms(BaseScalarField))
    s_fields = list (s_fields)
    dums = coord_sys._dummies
    invmetric = matrix.subs(list(zip(s_fields, dums))).inv().subs(list(zip(dums, s_fields)))
    N = T.rank()
    lis = list(range(len(changes), N))
    em = []
    k = len(changes) -1
    l = 0

    for j in range(N):
        if j in changes:
           em.append(k)
           k -= 1
        else:
            em.append(lis[l])
            l += 1

    new_order = tuple(em)

    raised = T

    for i in changes:
        raised = tc(tp(invmetric, raised), (0, i +2))

        raised = permutedims(raised, new_order)

    return raised


def simplify_array(A, d):
    R = A.rank ()
    F = flatten(A)
    for i in range (len(F)):
        F[i] = simplify(F[i])
    t = tuple([d]* R)
    R = (Array(F, (d,)*R))

    return R


def Kulkarni_Nomizu_product(A, B):
    AB = tp(A, B)
    out = permutedims (AB, (0,2,1,3)) + permutedims (AB, (1,3,0,2)) -\
               permutedims (AB, (0,3,1,2)) - permutedims (AB, (1,2,0,3))
    return out


def print_components(T, d):
    perms = list(set(permutations(np.repeat(range(d),T.rank()),T.rank())))
    for perm in perms :
        print(perm, factor(T[perm]))

def partial_derivative(T, expr):
    coord_sys = expr.atoms(CoordSystem).pop()
    indices = list (range(coord_sys.dim))

    diffd = [T.applyfunc( lambda a: d(a))
                for d in coord_sys.base_vectors()]
    diffd = (Array(diffd, (coord_sys.dim,)*(T.rank() +1)))

    return ImmutableDenseNDimArray(diffd)

def covariant_derivative(T, expr, n_up):
    Ch = metric_to_Christoffel_2nd(expr)
    PD = partial_derivative(T, expr)
    for i in range (n_up):
        PD += tc(tp(Ch , T), (2, (3+ i )))
    for i in range (T.rank() - n_up):
        PD -= tc( tp(Ch, T), (0, (i +3+ n_up)))
    return ImmutableDenseNDimArray((PD))


def Lie_derivative_covariant(X, T, expr, n_up):
    LD = tc(tp(X, covariant_derivative (T, expr, n_up)), (0 ,1))
    for j in range (n_up):
         LD -= tc(tp(covariant_derivative (X, expr, 1), T) ,(0 ,2+ j))
    for j in range (T.rank()-n_up):
         LD += tc(tp(covariant_derivative (X, expr, 1), T), (1 ,3+ n_up + j))
    return LD


def Lie_derivative_partial (X, T, expr, n_up):
    LD = tc(tp(X, partial_derivative(T, expr)), (0,1))
    print(LD)
    for j in range (n_up):
        LD -= tc(tp(partial_derivative (X, expr), T),(0,2+ j))
    for j in range (T.rank() - n_up):
        LD += tc(tp(partial_derivative (X, expr), T), (1,3+ n_up +j))
    return LD

In [5]:
def metric_to_Ricci_scalar(expr):
    ricci = metric_to_Ricci_components(expr)
    
    scalar = tc(raise_indices(ricci, expr, (0,)), (0,1))
    
    return scalar
def metric_to_torsion_components(expr):
    Ch2 = metric_to_Christoffel_2nd(expr)
    
    torsion = Ch2 - permutedims(Ch2, (0,2,1))
    
    return ImmutableDenseNDimArray (torsion)
def metric_to_contorsion_components(expr):
    tor = lower_indices ( metric_to_torsion_components ( expr ) , expr , (0 ,) )
    
    cont2 = tor - permutedims(tor, (1,2,0)) + permutedims(tor, (2,0,1))
    cont = tp(Rational(1,2), cont2)
    
    return ImmutableDenseNDimArray(cont)
def metric_to_Einstein_components(expr):
    ricci = metric_to_Ricci_components(expr)
    metric = twoform_to_matrix(expr)
    scalar = metric_to_Ricci_scalar(expr)
    Rg = tp(Rational(1, 2), metric, scalar)
    einstein = ricci - Rg

    return ImmutableDenseNDimArray(einstein)


def metric_to_Kretschmann_scalar(expr):
    riemann = metric_to_Riemann_components (expr)

    rie_low = lower_indices(riemann, expr, (0,))
    rie_up = raise_indices(riemann, expr, (1,2,3))

    kretschmann = tc(tc(tc(tc(tp(rie_low, rie_up),
            (0,4)), (0,3)), (0,2)), (0,1))

    return kretschmann

In [6]:
i, n, m = symbols('i n m', integer=True)

In [7]:
phi, c, psi = symbols('phi c psi')

In [8]:
i, j, k, l, m  = symbols('i j k l m', cls=Idx)

In [9]:

S= Function('S')(GR4_r.t, GR4_r.x, GR4_r.y, GR4_r.z)

In [10]:
A0= Function('A0')
A1= Function('A1')
A2= Function('A2')
A3= Function('A3')
A4= Function('A4')
A5= Function('A5')

In [11]:
A=(A0, A1, A2, A3, A4, A5)
indices = [0, 1, 2, 3]
T= Array([GR4_r.dt])

In [12]:
phi = 0
for k in indices:
    m= exp(I*S*c**(2))*(A[k](GR4_r.t, GR4_r.x, GR4_r.y, GR4_r.z)*c**(-k))
    phi = phi +m

In [13]:
phi
psi = Array([phi, 0, 0, 0])
psi

[A0(t, x, y, z)*exp(I*c**2*S(t, x, y, z)) + A1(t, x, y, z)*exp(I*c**2*S(t, x, y, z))/c + A2(t, x, y, z)*exp(I*c**2*S(t, x, y, z))/c**2 + A3(t, x, y, z)*exp(I*c**2*S(t, x, y, z))/c**3, 0, 0, 0]

In [14]:
Lorentz = TensorIndexType('Lorentz', dummy_name='L', dim=4, eps_dim=4, metric_symmetry=0, metric_name='metric')
mu = TensorIndex('mu', Lorentz, is_up=False)
nu, rho = tensor_indices('nu, rho', Lorentz)
B=TensorHead('B', [Lorentz])
M = IndexedBase('M')
repl = {Lorentz: diag(1, -1, -1, -1)}
repl.update({B(mu): [t, x, y, z]})

In [15]:
partial_derivative(psi, T)

[[I*c**2*A0(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Subs(Derivative(S(_xi, x, y, z), _xi), _xi, t) + I*c*A1(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Subs(Derivative(S(_xi, x, y, z), _xi), _xi, t) + I*A2(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Subs(Derivative(S(_xi, x, y, z), _xi), _xi, t) + exp(I*c**2*S(t, x, y, z))*Subs(Derivative(A0(_xi, x, y, z), _xi), _xi, t) + I*A3(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Subs(Derivative(S(_xi, x, y, z), _xi), _xi, t)/c + exp(I*c**2*S(t, x, y, z))*Subs(Derivative(A1(_xi, x, y, z), _xi), _xi, t)/c + exp(I*c**2*S(t, x, y, z))*Subs(Derivative(A2(_xi, x, y, z), _xi), _xi, t)/c**2 + exp(I*c**2*S(t, x, y, z))*Subs(Derivative(A3(_xi, x, y, z), _xi), _xi, t)/c**3, 0, 0, 0], [I*c**2*A0(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Subs(Derivative(S(t, _xi, y, z), _xi), _xi, x) + I*c*A1(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Subs(Derivative(S(t, _xi, y, z), _xi), _xi, x) + I*A2(t, x, y, z)*exp(I*c**2*S(t, x, y, z))*Subs(Derivative(S(t, _xi, y, z), _xi), _xi, x) + exp(I*c**2*S